In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
import pandas as pd

In [4]:
from datetime import datetime, timedelta
 

In [64]:
#Task 1
KEYWORDS = ['парсинг', 'python', 'slack']

In [8]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [9]:
articles = soup.find_all('article', class_='post post_preview')
articles

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/owlofmacloud/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/648/71b/2bb/64871b2bb6ddc572c1e7d1cecd02fecf.jpg" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">owlofmacloud</span>
 </a>
 <span class="post__time">сегодня в 14:00</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/macloud/blog/556522/">Падение Slack 4 января 2021</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/company/macloud/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Блог компании Маклауд'); }" rel="nofollow" title="Вы не подписаны на этот 

In [41]:
for article in articles:
    article_id = article.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not article_id:
        continue
    article_id = int(article_id.split('_')[-1])  
    for keyword in KEYWORDS: #находим статью, с ключевым словом
        if keyword in article.text.lower():
            title_element = article.find('a', class_='post__title_link')
            date = soup.find('span', class_='post__time').text
            print(date, title_element.text, title_element.attrs.get('href'))

сегодня в 14:00 Падение Slack 4 января 2021 https://habr.com/ru/company/macloud/blog/556522/
сегодня в 14:00 Управляем контактами GPIO из C# .NET 5 в Linux на одноплатном компьютере Banana Pi M64 (ARM64) и Cubietruck (ARM32) https://habr.com/ru/company/vdsina/blog/555598/
сегодня в 14:00 «ФП на Python посредством Coconut!» |> print https://habr.com/ru/post/556548/
сегодня в 14:00 Pythonnet. Как запустить C# код из Python https://habr.com/ru/post/556230/


In [66]:
articles_df = pd.DataFrame()
for article in articles:
    article_id = article.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not article_id:
        continue
    article_id = int(article_id.split('_')[-1])  
    for keyword in KEYWORDS: #находим статью, с ключевым словом
        if keyword in article.text.lower():
            date = soup.find('span', class_='post__time').text.split(' ')
            if 'сегодня' == date[0]:
                date = datetime.today().date()
            elif 'вчера' == date[0]:
                date = (datetime.today().date() - datetime.timedelta(days = 1))
            title_element = article.find('a', class_='post__title_link')
            title = title_element.text   
            link = title_element.attrs.get('href')
            row = {'date': date, 'title': title, 'link': link}
            articles_df = pd.concat([articles_df, pd.DataFrame([row])])
            
articles_df

,date,title,link
0,2021-05-10,Падение Slack 4 января 2021,https://habr.com/ru/company/macloud/blog/556522/
0,2021-05-10,Управляем контактами GPIO из C# .NET 5 в Linux...,https://habr.com/ru/company/vdsina/blog/555598/
0,2021-05-10,«ФП на Python посредством Coconut!» |> print,https://habr.com/ru/post/556548/
0,2021-05-10,Pythonnet. Как запустить C# код из Python,https://habr.com/ru/post/556230/


In [73]:
#Task 1(дополнительная часть)

Full_text_df = pd.DataFrame()
for article in articles:
    article_id = article.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not article_id:
        continue
    article_id = int(article_id.split('_')[-1])  
    title_element_1 = article.find('a', class_='post__title_link')
    title_1 = title_element_1.text
    link_1 = title_element_1.attrs['href']
    full_req = requests.get(link_1)
    soup_1 = BeautifulSoup(full_req.text, 'html.parser')
    full_text = soup_1.find_all('div', class_ = 'post__text post__text-html post__text_v1')
    full_text_1 = soup_1.find_all('div', class_ = 'post__text post__text-html post__text_v2')
    date_1 = soup_1.find('span', class_='post__time').text.split(' ')
    if 'сегодня' == date_1[0]:
        date_1 = datetime.today().date()
    elif 'вчера' == date_1[0]:
        date_1 = (datetime.today().date() - datetime.timedelta(days = 1))
    for item in full_text:
        for keyword in KEYWORDS: #находим статью, с ключевым словом
            if keyword in item.text.lower():
                row = {'date': date_1, 'title': title_1, 'link': link_1, 'text': item}
                Full_text_df = pd.concat([Full_text_df, pd.DataFrame([row])])        
    for item in full_text_1:
        for keyword in KEYWORDS: #находим статью, с ключевым словом
            if keyword in item.text.lower():
                row = {'date': date_1, 'title': title_1, 'link': link_1, 'text': item.text}
                Full_text_df = pd.concat([Full_text_df, pd.DataFrame([row])])           

In [75]:
Full_text_df.reset_index()

,index,date,title,link,text
0,0,2021-05-10,Падение Slack 4 января 2021,https://habr.com/ru/company/macloud/blog/556522/,"[[], [], \n, [], \r\n4 января 2021 года для мн..."
1,0,2021-05-10,Управляем контактами GPIO из C# .NET 5 в Linux...,https://habr.com/ru/company/vdsina/blog/555598/,"[[], [], \n, [], \r\nКогда заходит речь про пр..."
2,0,2021-05-10,Инструменты для аудита CSS,https://habr.com/ru/company/macloud/blog/556520/,"[[], [], \n, [], \r\nАудит (здесь и далее слов..."


In [77]:
#Task 2

EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [78]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'


In [110]:
params = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9, en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Length': '31',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
    'sec-ch-ua-mobile': '?0',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0',
}
PAYLOAD = {
    'emailAddresses':EMAIL
}

In [111]:
res = requests.post(URL, headers=params, json=PAYLOAD)
res

<Response [200]>

In [121]:
res.json()

result = pd.DataFrame(res.json()['breaches'])
result

,16613,17609,17110,17670,13094,37177,16768,3176,41,2,3587,17009,12,3,13662,3520,3164,2961,15,3669
breachId,16613,17609,17110,17670,13094,37177,16768,3176,41,2,3587,17009,12,3,13662,3520,3164,2961,15,3669
site,verifications.io,vk.com,azcentral.com,wishbone.io,myheritage.com,forums.vkmonline.com,canva.com,parapa.mail.ru,dropbox.com,linkedin.com,rayli.com.cn,zynga.com,vk.com,adobe.com,netlog.com,globalreach.eu,cfire.mail.ru,cdprojektred.com,imesh.com,youku.com
recordsCount,677914246,42536910,705538,37475476,110041653,825654,137434010,5029003,68591031,158591429,4844179,216159495,110121799,152046506,52904632,921298,12880232,1871459,50913222,94244880
description,Big data e-mail verification platform verifica...,"At some time in 2020, the Russian social netwo...","At an unconfirmed date, online Arizona newspap...","In January 2020, the online poll website Wishb...","In October 2017, a customer database belonging...","At an unconfirmed date, the Russian-language m...","In May 2019, graphic-design site Canva's datab...","In July and August 2016, two criminals execute...",Cloud storage company Dropbox suffered a major...,"In 2012, online professional networking platfo...","On an unconfirmed date, Chinese gossip site Ra...","In September 2019, the game developer Zynga wa...",Popular Russian social networking platform VKo...,"In October of 2013, criminals penetrated Adobe...",Netlog (formerly known as Facebox and Bingbox)...,"In 2016, Global Reach Technology's database wa...","In July and August of 2016, two criminals carr...","In March 2016, CDProjektRed.com.com's forum da...","In June 2016, a cache of over 51 million user ...",Youku is a large Chinese video content company...
publishDate,2019-03-28T00:00:00Z,2020-05-21T00:00:00Z,2020-01-03T00:00:00Z,2020-05-28T00:00:00Z,2017-11-04T00:00:00Z,2021-02-11T00:00:00Z,2019-06-13T00:00:00Z,2017-02-14T00:00:00Z,2016-10-24T00:00:00Z,2016-10-21T00:00:00Z,2017-03-01T00:00:00Z,2019-10-17T00:00:00Z,2016-10-29T00:00:00Z,2016-10-21T00:00:00Z,2018-02-18T00:00:00Z,2017-03-15T00:00:00Z,2017-02-14T00:00:00Z,2017-01-31T00:00:00Z,2016-10-23T00:00:00Z,2017-03-24T00:00:00Z
statistics,"{'usernames': 0, 'passwords': 0, 'emails': 677...","{'usernames': 0, 'passwords': 0, 'emails': 205...","{'usernames': 0, 'passwords': 702971, 'emails'...","{'usernames': 31111127, 'passwords': 10648734,...","{'usernames': 0, 'passwords': 110041647, 'emai...","{'usernames': 825566, 'passwords': 825654, 'em...","{'usernames': 137304104, 'passwords': 61348974...","{'usernames': 5029000, 'passwords': 5029003, '...","{'usernames': 0, 'passwords': 68591031, 'email...","{'usernames': 0, 'passwords': 111975337, 'emai...","{'usernames': 4841886, 'passwords': 4833694, '...","{'usernames': 214648143, 'passwords': 13476279...","{'usernames': 0, 'passwords': 107425364, 'emai...","{'usernames': 0, 'passwords': 129430596, 'emai...","{'usernames': 0, 'passwords': 52902665, 'email...","{'usernames': 13492, 'passwords': 921298, 'ema...","{'usernames': 12880232, 'passwords': 12880222,...","{'usernames': 1871410, 'passwords': 1871457, '...","{'usernames': 50909015, 'passwords': 50913222,...","{'usernames': 0, 'passwords': 94238535, 'email..."


In [146]:
new = result.loc[['publishDate','site', 'description']].T
new

,publishDate,site,description
16613,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
17609,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
17110,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
17670,2020-05-28T00:00:00Z,wishbone.io,"In January 2020, the online poll website Wishb..."
13094,2017-11-04T00:00:00Z,myheritage.com,"In October 2017, a customer database belonging..."
37177,2021-02-11T00:00:00Z,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m..."
16768,2019-06-13T00:00:00Z,canva.com,"In May 2019, graphic-design site Canva's datab..."
3176,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
41,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
2,2016-10-21T00:00:00Z,linkedin.com,"In 2012, online professional networking platfo..."


In [11]:
#Task 2 (дополнительная)
WALL = 'https://api.vk.com/method/wall.get'
GROUP = 'netology'
VERSION = '5.84'
SLEEP = 0.33
TOKEN = Token

In [12]:
PARAMS ={
    'owner_id': '-30159897',
    'domain': 'https://vk.com/netology',
    'count' : 50,
    'v' : VERSION,
    'access_token': TOKEN
}

In [13]:
res = requests.get(WALL, PARAMS)
res

<Response [200]>

In [17]:
wall_text_df = pd.DataFrame(res.json()['response']['items'])

In [34]:
wall_text_df['date'] = pd.to_datetime(wall_text_df['date'], unit="s")
wall_text_df[['date', 'text']]

,date,text
0,2021-04-28 13:30:00,"Работа мечты — та, на которой пригодятся все н..."
1,2021-05-11 08:11:00,Майские выходные — затратное время 🤑 \n \nНо э...
2,2021-05-10 15:43:00,Моушн-дизайн сейчас на пике популярности: его ...
3,2021-05-10 07:07:00,🤩 Запустили курс по основам Canva \n \nЗа шест...
4,2021-05-09 07:27:00,Искусственный интеллект умеет управлять автомо...
5,2021-05-08 08:49:00,С таким количеством новых фич Инстаграм станов...
6,2021-05-07 08:17:00,Пора отвлечься от огорода и шашлыков 👨‍🌾 \n\nС...
7,2021-05-06 16:01:00,Часто именно негативные убеждения мешают нам н...
8,2021-05-06 09:01:00,Спокойно отдыхаем и остаёмся в курсе всего с Н...
9,2021-05-05 09:37:00,🧶 18 мая стартует бесплатный интенсив «Как ста...
